In [1]:
!pip install opendatasets
!pip install kaggle
import opendatasets as od
import pandas as pd
from fastai.text.all import *


In [2]:
od.download("https://www.kaggle.com/competitions/word2vec-nlp-tutorial/")
!unzip /content/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
train = pd.read_csv("/content/labeledTrainData.tsv", header=0, \
                    delimiter="\t", quoting=3)

100%|██████████| 51.7M/51.7M [00:04<00:00, 12.8MB/s]



Extracting archive ./word2vec-nlp-tutorial/word2vec-nlp-tutorial.zip to ./word2vec-nlp-tutorial
Archive:  /content/word2vec-nlp-tutorial/labeledTrainData.tsv.zip
  inflating: labeledTrainData.tsv    


In [3]:
train

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.<b..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy Hines is a very entertaining film that obviously goes to great effort and lengths to faithfully recreate H. G. Wells' classic book. Mr. Hines succeeds in doing so. I, and those who watched his film with me, appreciated the fact that it was not the standard, predictable Hollywood fare that comes out every year, e.g. the Spielberg version with Tom Cruise that had only the slightest resemblance to the book. Obviously, everyone looks for different things in a movie. Those who envision themselves as amateur \""critics\"" look only to criticize everyth..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell) giving welcome investors (Robert Carradine) to Primal Park . A secret project mutating a primal animal using fossilized DNA, like ¨Jurassik Park¨, and some scientists resurrect one of nature's most fearsome predators, the Sabretooth tiger or Smilodon . Scientific ambition turns deadly, however, and when the high voltage fence is opened the creature escape and begins savagely stalking its prey - the human visitors , tourists and scientific.Meanwhile some youngsters enter in the restricted area of the security center and are attacked by a pack ..."
3,"""3630_4""",0,"""It must be assumed that those who praised this film (\""the greatest filmed opera ever,\"" didn't I read somewhere?) either don't care for opera, don't care for Wagner, or don't care about anything except their desire to appear Cultured. Either as a representation of Wagner's swan-song, or as a movie, this strikes me as an unmitigated disaster, with a leaden reading of the score matched to a tricksy, lugubrious realisation of the text.<br /><br />It's questionable that people with ideas as to what an opera (or, for that matter, a play, especially one by Shakespeare) is \""about\"" should be a..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious 80's exploitation, hooray! The pre-credits opening sequences somewhat give the false impression that we're dealing with a serious and harrowing drama, but you need not fear because barely ten minutes later we're up until our necks in nonsensical chainsaw battles, rough fist-fights, lurid dialogs and gratuitous nudity! Bo and Ingrid are two orphaned siblings with an unusually close and even slightly perverted relationship. Can you imagine playfully ripping off the towel that covers your sister's naked body and then stare at her unshaven genitals..."
...,...,...,...
24995,"""3453_3""",0,"""It seems like more consideration has gone into the IMDb reviews of this film than went into the source.<br /><br />Here's a review without pretensions:<br /><br />Just when you think nothing is going to happen, it doesn't.<br /><br />Dress it up any way you like, this is a dull film, full of unengaging characters doing very little of interest.<br /><br />One to put on if you want to convince an impressionable emo chick that you're like, so deep, man.<br /><br />Not something to watch for your own pleasure though.<br /><br />Unless.<br /><br />You're.<br /><br />Pretentious."""
24996,"""5064_1""",0,"""I don't believe they made this film. Completely unnecessary. The first film was okay. But there was no need for a sequel, certainly not after a television series that was already a sequel to the first film. This film feels 

In [4]:
dls = TextDataLoaders.from_df (train, path='.', valid_pct=0.1, seed=12345678,
                          text_col=2,  is_lm=True, tok_text_col='text')

In [5]:
dls.show_batch()

,text,text_
0,"xxbos "" this movie features two of my favorite actors in xxmaj kilmer and xxmaj downey . xxmaj it also boasts the always enjoyable xxmaj larry xxmaj miller in a too - small part . xxmaj despite this i found it to be nearly unwatchable . xxmaj michelle xxmaj monaghan may be pretty but she is nearly charisma free and the reasons for xxmaj downey 's character 's obsession with her character",""" this movie features two of my favorite actors in xxmaj kilmer and xxmaj downey . xxmaj it also boasts the always enjoyable xxmaj larry xxmaj miller in a too - small part . xxmaj despite this i found it to be nearly unwatchable . xxmaj michelle xxmaj monaghan may be pretty but she is nearly charisma free and the reasons for xxmaj downey 's character 's obsession with her character is"
1,"he 's a skinny sociopathic pedophile living in xxmaj xxunk the heavens , whichever ) , with a bunch of kids who work harder than the one 's in xxmaj kathy xxmaj lee xxmaj gifford 's sweat shops . xxmaj they sing oh - so - cute traditional songs of their xxunk while wearing clothing so stereotypical that i was surprised there was n't a little african - american boy in black","'s a skinny sociopathic pedophile living in xxmaj xxunk the heavens , whichever ) , with a bunch of kids who work harder than the one 's in xxmaj kathy xxmaj lee xxmaj gifford 's sweat shops . xxmaj they sing oh - so - cute traditional songs of their xxunk while wearing clothing so stereotypical that i was surprised there was n't a little african - american boy in black face"
2,". xxmaj during the film , there is a group of people walked out . xxmaj after the movie , many people said , "" that 's it ? "" xxmaj frankly speaking , i can not believe that this movie was awarded the best children film award . xxmaj if you are thinking of watching this film , i strongly recommend you not to . xxmaj you will regret it .","xxmaj during the film , there is a group of people walked out . xxmaj after the movie , many people said , "" that 's it ? "" xxmaj frankly speaking , i can not believe that this movie was awarded the best children film award . xxmaj if you are thinking of watching this film , i strongly recommend you not to . xxmaj you will regret it . xxmaj"
3,"thrilling performances with xxmaj jaffe very well in the title role and xxmaj cianelli very sinister as xxmaj guru . xxmaj rating- 10 out of 10 . "" xxbos "" never cast models and xxmaj playboy bunnies in your films ! xxmaj bob xxmaj fosse 's "" star 80 "" about xxmaj dorothy xxmaj stratten , of whom xxmaj bogdanovich was obsessed enough to have married her xxup sister after her murder","performances with xxmaj jaffe very well in the title role and xxmaj cianelli very sinister as xxmaj guru . xxmaj rating- 10 out of 10 . "" xxbos "" never cast models and xxmaj playboy bunnies in your films ! xxmaj bob xxmaj fosse 's "" star 80 "" about xxmaj dorothy xxmaj stratten , of whom xxmaj bogdanovich was obsessed enough to have married her xxup sister after her murder at"
4,"to mount another project , xxmaj i 'm delighted to say it was worth the wait . xxmaj cypher is a fascinating exploration of one man 's place in the world , and how through a completely logical chain of events , finds himself in a situation beyond his control . \n\n i do n't want to reveal too much about the plot , because one of the joys of xxmaj cypher","mount another project , xxmaj i 'm delighted to say it was worth the wait . xxmaj cypher is a fascinating exploration of one man 's place in the world , and how through a completely logical chain of events , finds himself in a situation beyond his control . \n\n i do n't want to reveal too much about the plot , because one of the joys of xxmaj cypher is"
5,"when she arrives at the location , a cowboy tells her she can give one last good - bye to her child . xxmaj one of the best segments ! xxup tour xxup eiffel - two mimes who fall in love could have been great , but , even though it has some nice cinema

In [6]:
learn = language_model_learner(dls, AWD_LSTM,
                               metrics=[accuracy,
                                        Perplexity()],
                               path="/content", wd=0.1).to_fp16()

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [7]:
learn.fit_one_cycle(5, 1e-2)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.417814,4.151760,0.280416,63.545750,04:50
1,4.395011,4.151170,0.280578,63.508278,04:48
2,4.367046,4.104181,0.284831,60.593086,04:50
3,4.289701,4.055780,0.289312,57.730171,04:49
4,4.214046,4.042590,0.290867,56.973694,04:49


In [8]:
learn.save('5epoch_text')

Path('/content/models/5epoch_text.pth')

In [10]:
learn = learn.load('5epoch_text')

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device)


In [11]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.044126,3.940119,0.301292,51.424709,05:17
1,3.978853,3.893672,0.305462,49.090805,05:17
2,3.930882,3.863135,0.308306,47.614372,05:17
3,3.900854,3.840229,0.311160,46.536144,05:18
4,3.842918,3.822260,0.313116,45.707409,05:17
5,3.774662,3.801499,0.315572,44.768227,05:17
6,3.725445,3.782579,0.317439,43.929188,05:14
7,3.678554,3.770488,0.319708,43.401230,05:12
8,3.638481,3.766406,0.320134,43.224438,05:12
9,3.601044,3.765929,0.320418,43.203812,05:13


In [12]:
learn.save_encoder('finetuned_text')

In [13]:
TEXT = "This movie is incredible, because"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75)
         for _ in range(N_SENTENCES)]

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/usr/local/lib/python3.10/dist-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

In [14]:
preds

["This movie is incredible , because it 's not nearly as good as one of Peter Sellers 's best movies , but it has n't been done before ( maybe it 's the same ! ) . The film was set in a",
 "This movie is incredible , because it features thousands of actual actresses in the film . Also , I 've seen many of the other actresses and actresses ( and one would say that the cast is better than most of them ) ."]

In [16]:
dls_clas = TextDataLoaders.from_df(train , seed= 1234567899, text_col=2, text_vocab=dls.vocab)
dls_clas.show_batch()

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)


/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)


,text,category
0,"xxbos "" match 1 : xxmaj tag xxmaj team xxmaj table xxmaj match xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley vs xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit xxmaj bubba xxmaj ray and xxmaj spike xxmaj dudley started things off with a xxmaj tag xxmaj team xxmaj table xxmaj match against xxmaj eddie xxmaj guerrero and xxmaj chris xxmaj benoit . xxmaj according to the rules of the match , both opponents have to go through tables in order to get the win . xxmaj benoit and xxmaj guerrero heated up early on by taking turns hammering first xxmaj spike and then xxmaj bubba xxmaj ray . a xxmaj german suplex by xxmaj benoit to xxmaj bubba took the wind out of the xxmaj dudley brother . xxmaj spike tried to help his brother , but the referee restrained him while xxmaj benoit and xxmaj guerrero",1
1,"xxbos "" raising xxmaj victor xxmaj vargas : a xxmaj review \n\n xxmaj you know , xxmaj raising xxmaj victor xxmaj vargas is like sticking your hands into a big , steaming bowl of oatmeal . xxmaj it 's warm and gooey , but you 're not sure if it feels right . xxmaj try as i might , no matter how warm and gooey xxmaj raising xxmaj victor xxmaj vargas became i was always aware that something did n't quite feel right . xxmaj victor xxmaj vargas suffers from a certain xxunk on the director 's part . xxmaj apparently , the director thought that the ethnic backdrop of a xxmaj latino family on the lower east side , and an idyllic storyline would make the film critic proof . xxmaj he was right , but it did n't fool me . xxmaj raising xxmaj victor xxmaj vargas is",0
2,"xxbos "" okay , so xxmaj i 'm not a big video game buff , but was the game xxmaj house of the xxmaj dead really famous enough to make a movie from ? xxmaj sure , they went as far as to actually put in quick video game clips throughout the movie , as though justifying any particular scene of violence , but there are dozens and dozens of games that look exactly the same , with the hand in the bottom on the screen , supposedly your own , holding whatever weapon and goo - ing all kinds of aliens or walking dead or snipers or whatever the case may be . \n\n xxmaj it 's an interesting premise in xxmaj house of the xxmaj dead , with a lot of college kids ( loaded college kids , as it were , kids who are able to pay",0
3,"xxbos "" myra xxup breckinridge is one of those rare films that established its place in film history immediately . xxmaj praise for the film was absolutely nonexistent , even from the people involved in making it . xxmaj this film was loathed from day one . xxmaj while every now and then one will come across some maverick who will praise the film on philosophical grounds ( aggressive feminism or the courage to tackle the issue of xxunk ) , the film has not developed a cult following like some notorious flops do . xxmaj it 's not hailed as a misunderstood masterpiece like xxup scarface , or trotted out to be ridiculed as a camp classic like xxup showgirls . \n\n xxmaj undoubtedly the reason is that the film , though outrageously awful , is not lovable , or even likable . xxup myra xxup breckinridge is just",0
4,"xxbos "" by 1987 xxmaj hong xxmaj kong had given the world such films as xxmaj sammo xxmaj hung 's ` encounters of the xxmaj spooky xxmaj kind ' xxmaj chow xxmaj yun xxmaj fat in xxmaj john xxmaj woo 's iconic ` a xxmaj better xxmaj tomorrow ' , ` zu xxmaj warriors ' and the classic ` mr xxmaj vampire ' . xxmaj jackie xxmaj chan was having international success on video , but it was with ` a xxmaj chinese xxmaj ghost xxmaj story ' that xxup hk cinema had its first real crossover theatrical hit in the xxmaj west for many years . \n\n xxmaj western filmgoers had never seen anything like it . xxmaj it was a film that took various ingredients that xxup hk cinema had used for years ( flying swordsman , wildly choreographed martial arts and the supernatural ) and blended them",1
5,"xxbos "" prior to this release , xxmaj neil labute had this

In [17]:
def train_encoder(learn, iterations):
  learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
  learn = learn.load_encoder('finetuned_text')
  learn.fit_one_cycle(iterations, 2e-2)
  learn.freeze_to(-2)
  learn.fit_one_cycle(iterations, slice(1e-2/(2.6**4),1e-2))
  learn.freeze_to(-3)
  learn.fit_one_cycle(iterations, slice(5e-3/(2.6**4),5e-3))
  learn.unfreeze()
  learn.fit_one_cycle(iterations+1, slice(1e-3/(2.6**4),1e-3))
  return learn

In [18]:
learn = train_encoder(learn, 3)

/usr/local/lib/python3.10/dist-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

epoch,train_loss,valid_loss,accuracy,time
0,0.291214,0.246461,0.900600,01:55
1,0.266692,0.231082,0.913000,01:54
2,0.242944,0.215061,0.917800,01:53


/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels 

epoch,train_loss,valid_loss,accuracy,time
0,0.257580,0.220841,0.914400,02:15
1,0.212685,0.194584,0.924000,02:15
2,0.180083,0.187017,0.932200,02:15


/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels 

epoch,train_loss,valid_loss,accuracy,time
0,0.200062,0.185728,0.931000,03:43
1,0.160015,0.182853,0.933400,03:43
2,0.086966,0.192214,0.935200,03:42


/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels 

epoch,train_loss,valid_loss,accuracy,time
0,0.078752,0.203620,0.933800,04:54
1,0.072578,0.220409,0.933000,04:55
2,0.046236,0.236707,0.934800,04:55
3,0.032817,0.242645,0.932600,04:55


/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  o = r[c] if isinstance(c, int) or not c in getattr(r, '_fields', []) else getattr(r, c)
/usr/local/lib/python3.10/dist-packages/fastai/data/transforms.py:214: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels 

In [22]:
learn.save('double_tuned_text')

Path('models/double_tuned_text.pth')